In [1]:
import os
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')

DATABASE_PATH: database/db/ohlcv_data.db


In [6]:
import sqlite3
conn = sqlite3.connect(database_path)
conn.execute("SELECT * FROM market")
conn.fetchone()


: 

In [2]:
import pandas as pd
import time
from utils.flows.fetch_calculate_insert import fetch_calculate_and_insert
from utils.calculation.indicators import calculate_ema, calculate_supertrend, calculate_spike, detect_large_gap, calculate_average_volume, calculate_exponential_regression
from utils.calculation.supertrend import faster_supertrend
from utils.calculation.slope_r2 import calculate_exponential_regression_optimized
from utils.calculation.optimized_indicators import calculate_spike_optimized, detect_large_gap_optimized, calculate_average_volume_optimized
from data.fetch.indian_equity import fetch_symbol_list_indian_equity
from dotenv import load_dotenv
import os
import traceback

# Load environment variables from .env file
load_dotenv(dotenv_path='config/.env')

# Access the DATABASE_PATH environment variable
database_path = os.getenv('DATABASE_PATH')

def calculate_technical_indicators(market_name, timeframe='1d', symbol_list=None, all_entries=True, start_timestamp=None):
    '''
    One time run function for calculating all custom indicators for a given market.
    Calculates and saves the indicator results into database.
    '''
    try:
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_ema, length=100)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_ema, length=200)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=faster_supertrend, period=7, multiplier=3)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_exponential_regression_optimized, window=90)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_exponential_regression_optimized, window=30)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_exponential_regression_optimized, window=15)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_spike_optimized, lookback_period=90, spike_threshold=0.15)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=detect_large_gap_optimized, lookback_period=90, gap_threshold=0.15)
        fetch_calculate_and_insert(market_name=market_name, timeframe=timeframe, start_timestamp=start_timestamp, symbol_list=symbol_list, all_entries=all_entries, calculation_func=calculate_average_volume_optimized, lookback_period=90)
    except Exception as e:
        print(f"Error calculating technical indicators: {e}")
        print(f"Full traceback:")
        print(traceback.format_exc())


/usr/local/lib/python3.11/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [ ]:
symbol_list = fetch_symbol_list_indian_equity(complete_list=False)
all_entries = False
calculate_technical_indicators(market_name='indian_equity', timeframe='1d', symbol_list=symbol_list, all_entries=all_entries, start_timestamp=start_timestamp)

In [4]:
from utils.db.fetch import fetch_ohlcv_data
from data.fetch.indian_equity import fetch_symbol_list_indian_equity
symbol_list = fetch_symbol_list_indian_equity(complete_list=False)
all_entries = False
ohlcv = fetch_ohlcv_data(market_name='indian_equity', timeframe='1d',symbol_list=symbol_list, all_entries=all_entries, start_timestamp=start_timestamp)

In [3]:
import pandas as pd
current_timestamp = pd.Timestamp.now()
start_timestamp = current_timestamp - pd.Timedelta(days=500)
start_timestamp = start_timestamp.strftime('%Y-%m-%d %H:%M:%S')
start_timestamp

'2023-05-19 14:54:02'

In [ ]:
calculate_technical_indicators(market_name='indian_equity', timeframe='1d', start_timestamp=start_timestamp)